In [4]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt/'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util


In [2]:
def compare_vafs(maf, referenceCohortMaf):
    
    returnDict = {'allMutations': [], 'allOncogenicMutations':[],
                  'allRecurrentTumorSuppressors': [], 'oncogenicRecurrentTumorSuppresors': [],
                  'allRecurrentOncogenes': [], 'oncogenicRecurrentOncogenes': []}
    
    recurrentTumorSupressors, recurrentOncogenes = maf_analysis_utils.enumerate_recurrently_mutated_tumor_supressors_and_oncogenes(referenceCohortMaf, thresh=.05)
    oncogenicMutColNames = set(['Likely Oncogenic', 'Oncogenic', 'Predicted Oncogenic'])
    cases = set(maf['Tumor_Sample_Barcode'])
    genes = set(maf['Hugo_Symbol'])
    
    print recurrentOncogenes
    
    for case in cases:
        print case
        #for gene in genes:
         #   geneMaf = maf[maf['Hugo_Symbol'] == gene]
            #here are all the cases/areas we look at
        caseMafAllMuts = maf[maf['Tumor_Sample_Barcode'] == case]
        #caseMafAllMuts = geneMaf[geneMaf['Tumor_Sample_Barcode'] == case]
        
        caseMafAllOncogenicMuts = caseMafAllMuts[caseMafAllMuts['oncogenic'].isin(oncogenicMutColNames)]
        caseMafAllReccurentTumorSuppressors = caseMafAllMuts[caseMafAllMuts['Hugo_Symbol'].isin(recurrentTumorSupressors)]
        caseMafOncogenicRecurrentTumorSuppresors = caseMafAllReccurentTumorSuppressors[caseMafAllReccurentTumorSuppressors['oncogenic'].isin(oncogenicMutColNames)]
        caseMafAllRecurrentOncogenes = caseMafAllMuts[caseMafAllMuts['Hugo_Symbol'].isin(recurrentOncogenes)]
        caseMafOncogenicRecurrentOncogenes = caseMafAllRecurrentOncogenes[caseMafAllRecurrentOncogenes['oncogenic'].isin(oncogenicMutColNames)]
        
        #add information to our return dict
        returnDict['allMutations'] = returnDict['allMutations'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafAllMuts['ccf_Mcopies'])))
        returnDict['allOncogenicMutations'] = returnDict['allOncogenicMutations'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafAllOncogenicMuts['ccf_Mcopies'])))
        returnDict['allRecurrentTumorSuppressors'] = returnDict['allRecurrentTumorSuppressors'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafAllReccurentTumorSuppressors['ccf_Mcopies'])))
        returnDict['oncogenicRecurrentTumorSuppresors'] = returnDict['oncogenicRecurrentTumorSuppresors'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafOncogenicRecurrentTumorSuppresors['ccf_Mcopies'])))
        returnDict['allRecurrentOncogenes'] = returnDict['allRecurrentOncogenes'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafAllRecurrentOncogenes['ccf_Mcopies'])))
        returnDict['oncogenicRecurrentOncogenes'] = returnDict['oncogenicRecurrentOncogenes'] + analysis_utils.calculate_all_pairwise_differences(np.array(list(caseMafOncogenicRecurrentOncogenes['ccf_Mcopies'])))
        
    return returnDict

In [1]:
def create_two_hit_mutation_summary(df, alleleCountThresh =5, doubleThresh=1.75):
    
    tumorSuppressors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
    
    listOfDicts = []
    #PROCESS THE MAF AS NEEDED
    df = maf_analysis_utils.fix_mll_genes(df)
    df = maf_analysis_utils.mark_cases_with_flat_genomes(df)
    df = maf_analysis_utils.mark_cases_with_median_clonal_vaf_of_case(df)
    df = df[df['oncogenic'].notnull()]
    df = df[df['HGVSp_Short'].notnull()]
    
    df['isDouble'] = df.apply(lambda row: clonality_analysis_util.is_mut_double_hit(row, row['FlatGenome'], doubleThresh), axis=1)
                                                                              
    dictOfMuts = dict(Counter(df[df['isDouble'] == True]['Hugo_Symbol']))
    for gene, count in dictOfMuts.items():
        if count > 1:
            for allele, aCount in Counter(df[(df['Hugo_Symbol'] == gene) & (df['isDouble'] == True)]['HGVSp_Short']).items():
                indel = False
                if '*' in allele: indel = True
                mutType = 'oncogene'
                if gene in tumorSuppressors: mutType = 'tumor_suppresor'
                d = {'Gene': gene, 'Hugo_Symbol':gene, 'Allele': allele, 'AlleleCount': aCount, 'count': count, 'orderingVal': count, 'indel':indel, 'mutType': mutType}
                listOfDicts.append(d)
        else: #PROCESS DATA SEPARATELY FOR ONE OFF ALELLES
            allele = df[(df['Hugo_Symbol'] == gene) & (df['isDouble'] == True)].iloc[0]['HGVSp_Short']
            indel = False
            if '*' in allele: indel = True
            mutType = 'oncogene'
            if gene in tumorSuppressors: mutType = 'tumor_suppresor'
            d = {'Gene': 'other', 'Hugo_Symbol':gene, 'Allele': allele, 'AlleleCount': 1, 'count': 1, 'orderingVal': -1, 'indel':indel, 'mutType': mutType}
            listOfDicts.append(d)
    
    df = pd.DataFrame(listOfDicts)
    df['label'] = df.apply(lambda row: row['Allele'] if row['AlleleCount'] >= alleleCountThresh and row['Allele'] != 'various' else None, axis=1)
    return df

In [3]:
colorectalDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/subsettedMafs/Colorectal_HypermutantCaseMuts_MAF_ANNO_trinuc.maf')
endometrialDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/subsettedMafs/Endometrial_HypermutantCaseMuts_MAF_ANNO_trinuc.maf')
gliomaDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/subsettedMafs/Glioma_HypermutantCaseMuts_MAF_ANNO_trinuc.maf')


NameError: name 'pd' is not defined

In [109]:
endoSummary = create_two_hit_mutation_summary(endometrialDf)
colorectalSummary =  create_two_hit_mutation_summary(colorectalDf)
gliomaSummary =  create_two_hit_mutation_summary(gliomaDf)

In [58]:
endoSummary.to_csv('~/Desktop/WORK/dataForLocalPlotting/endometrialDoubleHitPlotting.tsv', index=False, sep='\t')
colorectalSummary.to_csv('~/Desktop/WORK/dataForLocalPlotting/colorectalDoubleHitPlotting.tsv', index=False, sep='\t')
gliomaSummary.to_csv('~/Desktop/WORK/dataForLocalPlotting/gliomaDoubleHitPlotting.tsv', index=False, sep='\t')

In [67]:
def summarize_gene_mut_features_of_double_hit_muts(df):
    tumorSuppressors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
    df = df[df['Gene'] != 'other']
    nTumorSuppresors = df[df['Hugo_Symbol'].isin(tumorSuppressors)].shape[0]
    print 'Oncogenes ', df.shape[0] - nTumorSuppresors, 'Tumor Suppressors: ', nTumorSuppresors
    print Counter(df[~df['Hugo_Symbol'].isin(tumorSuppressors)]['Hugo_Symbol'])


In [5]:
combinedDf = pd.concat([endoSummary, colorectalSummary])
combinedDf.to_csv('~/Desktop/WORK/dataForLocalPlotting/combinedDoubleHitPlotting.tsv', index=False, sep='\t')

NameError: name 'endoSummary' is not defined

In [6]:
endoColorectalMaf = pd.concat([colorectalDf, endometrialDf])
endoColorectalMaf['quadNuc'] = endoColorectalMaf.apply(lambda row: mutationSigUtils.create_reference_four_nuc(row['Ref_Tri'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


In [141]:
def summarize_mutation_motif_info_of_mvish_muts(maf, df):
    
    poleMotifs = set(['TCAT', 'TCAA', 'TCTG', 'TCTT'])
    
    listOfDicts = []
    for hgvs in set(df['Allele']):
        quadNucs = maf[(maf['HGVSp_Short'] == hgvs) & (maf['isDouble'] == True)]['quadNuc']
        for q in quadNucs:
            if q == None:
                listOfDicts.append({'motif': 'MSI indel', 'orderingVal': 1})
            elif q in poleMotifs:
                listOfDicts.append({'motif': 'POLE', 'orderingVal': 2})
            else:
                listOfDicts.append({'motif': 'other', 'orderingVal': 3})
    return pd.DataFrame(listOfDicts)


In [7]:
endoColorectalMaf = maf_analysis_utils.fix_mll_genes(endoColorectalMaf)
endoColorectalMaf = maf_analysis_utils.mark_cases_with_flat_genomes(endoColorectalMaf)
endoColorectalMaf = maf_analysis_utils.mark_cases_with_median_clonal_vaf_of_case(endoColorectalMaf)
endoColorectalMaf = endoColorectalMaf[endoColorectalMaf['oncogenic'].notnull()]
endoColorectalMaf = endoColorectalMaf[endoColorectalMaf['HGVSp_Short'].notnull()]
    
endoColorectalMaf['isDouble'] = endoColorectalMaf.apply(lambda row: clonality_analysis_util.is_mut_double_hit(row, row['FlatGenome'], 1.75), axis=1)
    

/Users/friedman/Desktop/mnt//ifs/work/taylorlab/friedman/myUtils/maf_analysis_utils.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  caseMaf['clonal'] = caseMaf['ccf_expected_upper_em'].apply(lambda x: 1 if x > clonalThresh else 0)
/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1019: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [142]:
df = summarize_mutation_motif_info_of_mvish_muts(endoColorectalMaf, combinedDf)


In [144]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/MVISH_by_motif.tsv', index=False, sep='\t')

**Do all impact summaries of double spot mutation** <br/>  <br/>

In [14]:
endoColorectalMaf['allele'] = endoColorectalMaf.apply(lambda row: row['Hugo_Symbol'] + '_' + row['HGVSp_Short'], axis=1)
observedDoubleMuts = Counter(endoColorectalMaf[endoColorectalMaf['isDouble'] == True]['allele'])
print observedDoubleMuts

Counter({'RNF43_p.G659Vfs*41': 44, 'TCF7L2_p.K468Sfs*23': 14, 'MSH3_p.K383Rfs*32': 12, 'JAK1_p.K860Nfs*16': 10, 'PTEN_p.K267Rfs*9': 6, 'TGFBR2_p.K153Afs*3': 5, 'INPPL1_p.R1156Gfs*46': 4, 'ASXL1_p.G645Vfs*58': 3, 'KMT2C_p.X2481_splice': 2, 'TP53BP1_p.X1769_splice': 2, 'B2M_p.V69Wfs*34': 2, 'APC_p.T1556Nfs*3': 2, 'KMT2D_p.X2683_splice': 2, 'FLCN_p.H429Tfs*39': 2, 'ERF_p.G299Efs*12': 1, 'HLA-A_p.W157*': 1, 'EP300_p.H2324Tfs*29': 1, 'TGFBR2_p.K153Sfs*35': 1, 'APC_p.R856Nfs*6': 1, 'NF1_p.R1362*': 1, 'FUBP1_p.S11Lfs*43': 1, 'CYLD_p.N719Mfs*13': 1, 'ASXL2_p.X135_splice': 1, 'SOCS1_p.P36Vfs*75': 1, 'CIC_p.P1248Hfs*54': 1, 'NF1_p.N78Ifs*7': 1, 'ARID1A_p.P224Rfs*8': 1, 'ATM_p.K2811Sfs*46': 1, 'TSC1_p.R420Gfs*20': 1, 'KMT2C_p.K2797Rfs*26': 1, 'CREBBP_p.X608_splice': 1, 'MSH6_p.F1088Lfs*5': 1, 'APC_p.R2204*': 1, 'TP53_p.V73Wfs*50': 1, 'PIK3R1_p.R348*': 1, 'RAD50_p.K722Rfs*14': 1, 'PTEN_p.R130Q': 1, 'MSH6_p.E1322*': 1, 'MGA_p.E1249Rfs*42': 1, 'HNF1A_p.P291Qfs*51': 1, 'HLA-B_p.X25_splice': 1, 'NF1_p